# Solving Mixed integer Linear Programs with Benders and D-wave Quantum Annealing


### This is a tutorial done in the context of the paper "A hybrid Quantum-Classical Algorithm for Mixed-Integer Optimization in Power Systems".

In this code tutorial, we demonstrate how to solve a subset of Mixed Integer Linear Programs with Benders and D-Wave quantum Annealers. This tutorial provides a general explanation of the process that must be followed, using the test case of Optimal Transmition Switching. Moreover, the Benders acceleration techniques, that have been presented in the paper, are included in this tutorial.


In [ ]:
#Import Necessary Libraries
import pandas as pd
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt 
from numpy.linalg import inv
import math
import matlab.engine
from pyomo.environ import *
from gurobipy import GRB
import gurobipy as gp
import dimod
from dimod import ConstrainedQuadraticModel,CQM,Binary,quicksum,ExactCQMSolver
from dwave.system import LeapHybridCQMSampler
import re
from dwave.system import LeapHybridCQMSampler
from docplex.mp.model_reader import ModelReader
from qiskit.exceptions import QiskitError
from qiskit_optimization import QuadraticProgram
import matplotlib.pyplot as plt
from docplex.mp.model import Model
from qiskit_optimization.translators import from_docplex_mp, from_gurobipy
import matlab.engine

In [ ]:
maxpeak = 1.2

In [ ]:
eng = matlab.engine.start_matlab()

#Load the 6 bus case 
content = eng.load("./cases/case_6ww.mat",nargout=1)

#Uncomment to test for IEEE-14 Bus system
# content = eng.load("case14.mat",nargout=1)

eng.quit()

#### Load system data

In [ ]:
ERSE_ANO_COMPLETO = pd.read_excel('cases\ERSE_MTE.xlsx')
ERSE_ANO_COMPLETO = np.array(ERSE_ANO_COMPLETO)
ERSE_ANO_COMPLETO = np.append(ERSE_ANO_COMPLETO, ERSE_ANO_COMPLETO[0])
ERSE_ANO_COMPLETO_AV = np.mean(ERSE_ANO_COMPLETO.reshape(-1, 4), axis=1)
ERSE_ANO_COMPLETO_AV = ERSE_ANO_COMPLETO_AV[0:8736]
ERSE_DIA_MEDIA = np.array([sum(ERSE_ANO_COMPLETO_AV[i::364]) for i in range(len(ERSE_ANO_COMPLETO_AV) // 365)])
ERSE_DIA_MEDIA = np.append(ERSE_DIA_MEDIA, ERSE_DIA_MEDIA[0])
ERSE_DIA_MEDIA = ERSE_DIA_MEDIA/np.mean(ERSE_DIA_MEDIA)
df = pd.DataFrame (ERSE_ANO_COMPLETO_AV)
df.to_excel('cases\ERSE_ANO_COMPLETO_AV.xlsx', index=False)
df = pd.DataFrame (ERSE_DIA_MEDIA)
df.to_excel('cases\ERSE_DIA_MEDIA.xlsx', index=False)

In [ ]:
MPCbaseMVA = np.asarray(content['mpc']['baseMVA'])
MPCbus = np.asarray(content['mpc']['bus'])
MPCgen = np.asarray(content['mpc']['gen'])
MPCbranch = np.asarray(content['mpc']['branch'])
MPCgencost_check = np.asarray(content['mpc']['gencost'])

In [ ]:
NL = len(MPCbranch)
NB = len(MPCbus)
NGENS = len(MPCgencost_check)

In [ ]:
##################### IMPORTANT ##############################
greenflag=1
for i in range(0,NGENS):
    if(MPCgencost_check[i][0]==1):
        print("Generator ", str(i+1), " is NOT SUPPORTED!!!")
        greenflag=0
if greenflag:
    print("GOOD TO GO!")
else:
    print("NO GO!")

In [ ]:
costcoeff = np.zeros((NGENS,3), dtype=float)
for gen in range(0,NGENS): 
    NCoeff = MPCgencost_check[gen][3]
    if(NCoeff==1):
        costcoeff[gen][0] = MPCgencost_check[gen][4]
    elif (NCoeff==2):
        costcoeff[gen][1] = MPCgencost_check[gen][4]
        costcoeff[gen][0] = MPCgencost_check[gen][5]
    elif (NCoeff==3):
        costcoeff[gen][2] = MPCgencost_check[gen][4]
        costcoeff[gen][1] = MPCgencost_check[gen][5]
        costcoeff[gen][0] = MPCgencost_check[gen][6]
    

In [ ]:
column_names = ["Generator", "Bus", "a", "b", "c", "Pmin", "Pmax"]
dfGenerators = pd.DataFrame(columns = column_names)
dfGenerators['Generator'] = np.array([i for i in range(1,NGENS+1)])
dfGenerators['Bus'] = (MPCgen[:,0])
dfGenerators = dfGenerators.astype({"Bus": int})
dfGenerators['c'] = (costcoeff[:,0])
dfGenerators['b'] = (costcoeff[:,1])
dfGenerators['a'] = (costcoeff[:,2])
dfGenerators['Pmin'] = 0*(MPCgen[:,9])
dfGenerators['Pmax'] = (MPCgen[:,8])/6

In [ ]:
column_names = ["From", "To", "X", "PLimit"]
dfLines = pd.DataFrame(columns = column_names)
dfLines['From'] = (MPCbranch[:,0])
dfLines = dfLines.astype({"From": int})
dfLines['To'] = (MPCbranch[:,1])
dfLines = dfLines.astype({"To": int})
dfLines['X'] = (MPCbranch[:,3])
dfLines['PLimit'] = (MPCbranch[:,5])

In [ ]:
column_names = ["Time", "Bus", "PD"]
dfLoads = pd.DataFrame(columns = column_names)
dfLoads['Time'] = (np.ones(NB))
dfLoads = dfLoads.astype({"Time": int})
dfLoads['Bus'] = (MPCbus[:,0])
dfLoads = dfLoads.astype({"Bus": int})
dfLoads['PD'] = (MPCbus[:,2])/5

In [ ]:
gens = dfGenerators['Generator'].tolist()
costs = dfGenerators['b'].tolist()

In [ ]:
A_lines = np.zeros((NL,NB))
lines = np.zeros((NL,2), dtype=int)
lineFrom = dfLines['From'].to_numpy() -1
lineTo = dfLines['To'].to_numpy() -1
for l in range(0,NL):
    A_lines[l][lineFrom[l]] = 1
    A_lines[l][lineTo[l]] = -1 
    lines[l] = lineFrom[l], lineTo[l]

In [ ]:
Xlines = dfLines['X'].to_numpy()
Ydiag = np.diag(1/dfLines['X'].to_numpy())
LimitLines = dfLines['PLimit'].to_numpy()
Ydiag
Y = A_lines.transpose() @ Ydiag @ A_lines

In [ ]:
busGenData = []
busLoadData = np.empty((3,NB), dtype=object)
gonbusb = np.empty(NB, dtype=object)
for i in range(1,NB+1):
    genonbus = dfGenerators.loc[(dfGenerators['Bus'] == i)]
    gonbusb[i-1] = genonbus['Generator'].to_numpy()-1
    busGenData.append(genonbus)
    loadonbus = dfLoads.loc[(dfLoads['Time'] == 1) & (dfLoads['Bus'] == i)]
    busLoadData[0,i-1] = loadonbus
Xij = np.zeros((NB,NB), dtype=float)

In [ ]:
#Choose a load profile
T=1
profiler = np.array([0.54,0.50,0.47,0.46,0.46,0.48,
                     0.52,0.61,0.78,0.92,0.99,1.04,
                     1.16,1.22,1.10,1.03,1.00,0.95,
                     0.88,0.83,0.79,0.75,0.71,0.63])/2
busLoadsP = np.zeros((T,NB))
loadprofile = np.zeros((T,NB))
for t in range(0,T):
    for bus in range(0,NB):
        loadprofile[t][bus] = profiler[t]
        #loadprofile[t][bus] = 1
for t in range(0,T):
    for bus in range(0,NB):
        busLoadsP[t][bus] = loadprofile[t][bus]*sum(busLoadData[0][bus]['PD'])
busLoadsP

In [ ]:
fig, ax = plt.subplots()
ax.bar([t for t in range(T)], np.sum(busLoadsP,1))

#### Denoting the Mixed Integer Linear program we want to solve with a quantum annealer

It is worth noting that Optimal Transmition Switching is used as an EXAMPLE. We emphasise that the following gurobipy formulation of the Mixed Integer Linear Program (MILP) can be replaced by any MILP formulated by gurobipy in the form:

\begin{equation} \label{first_form}
\begin{aligned}
\max_{\boldsymbol{z},\boldsymbol{y}} \quad &\mathbf{i}^{T} \boldsymbol{z} + \mathbf{c}^{T} \boldsymbol{y} \\
    s.t. \quad &\mathbf{A} \boldsymbol{z} + \mathbf{B} \boldsymbol{y} \leq \mathbf{b} \\
    % \mathbf{C} \mathbf{z} \geq \mathbf{d} \\
     & {\boldsymbol{z}} \in \mathbb{Z}, {\boldsymbol{z}} \in \{0,1\}^n, \boldsymbol{y} \in \mathbb{R}^{p}_{+}
\end{aligned} 
\end{equation}

The following steps would be adapted to the formulation provided. 

In [ ]:
 def runOPF():
    model = gp.Model('DC-OPF_2')
    slackbus = 2 #Denote the slack bus
    times = np.array([t for t in range(0,T)]) #Denote number of Timeslots
    pgs = model.addVars(T,NGENS, lb=0, ub=9999.9, vtype=GRB.CONTINUOUS, name="PG") #Define the generators generation variable
    delta = model.addVars(T,NB, lb=-360, ub=360, vtype=GRB.CONTINUOUS, name="delta") #Define variable for voltage angles
    Pline = model.addVars(T,NL, lb=-999.9, ub=999.9, vtype=GRB.CONTINUOUS, name="PL") #Define variable for active power in the lines
    linestatus = model.addVars(T,NL, lb=0, ub=1, vtype=GRB.BINARY, name="linestatus") #Define variable for the operational status of the lines
    SBase=100

    M = 10000  # Set M to a sufficiently large value

    model.addConstrs(Pline[t, l] <= (delta[t, i] - delta[t, j]) / Xlines[l] + M*(1-linestatus[t,l])
                     for l, (i, j) in enumerate(lines)
                     for t in range(0, T))
    model.addConstrs(-Pline[t, l] <= -(delta[t, i] - delta[t, j]) / Xlines[l] + M*(1-linestatus[t,l])
                 for l, (i, j) in enumerate(lines)
                 for t in range(0, T))

    model.addConstrs(pgs[t,generator] <= dfGenerators['Pmax'][generator] 
                     for generator in range(0,NGENS)
                     for t in range(0,T));

    model.addConstrs(-pgs[t,generator] <= -dfGenerators['Pmin'][generator] 
                     for generator in range(0,NGENS)
                     for t in range(0,T));
    
    model.addConstrs(delta[t,slackbus]<=0 
                     for t in range(0,T));
    model.addConstrs(-delta[t,slackbus]<=0 
                     for t in range(0,T));

    model.addConstrs(delta[t,bus]<=360
                     for t in range(0,T)
                    for bus in range(NB));
    
    model.addConstrs(-delta[t,bus]<=360
                     for t in range(0,T)
                    for bus in range(NB));
    
    model.addConstrs(-Pline[t,l] 
                     <= linestatus[t,l] * LimitLines[l]
                     for l in range(0, NL)
                     for t in range(0,T));

    model.addConstrs(Pline[t,l] 
                     <= linestatus[t,l] * LimitLines[l] 
                     for l in range(0, NL)
                     for t in range(0,T));
    
    model.addConstrs(gp.quicksum([pgs[t,gen] for gen in gonbusb[bus]])
                        <= (busLoadsP[t][bus]
                           + gp.quicksum(A_lines[l][bus]*Pline[t,l] 
                                         for l in range(0,NL)))
                           for bus in range(0,NB) 
                           for t in range(0,T));
    
    model.addConstrs(-gp.quicksum([pgs[t,gen] for gen in gonbusb[bus]])
                        <= -(busLoadsP[t][bus]
                           + gp.quicksum(Pline[t,l] *A_lines[l][bus]
                                         for l in range(0,NL)))
                           for bus in range(0,NB) 
                           for t in range(0,T));
    
    #Change for simulation with 14 bus system
    model.addConstrs(gp.quicksum(linestatus[t, l] for l in range(NL))<=5 for t in range(T));
    
    totalgencost = gp.quicksum(gp.quicksum((pgs[t,gen]*dfGenerators['a'][gen]
                                + pgs[t,gen]*dfGenerators['b'][gen]) 
                               for gen in range(0,NGENS))
                              for t in range(0,T));
    
    model.setObjective(totalgencost,sense=GRB.MINIMIZE);
    model.write('junk.lp')

    model.update()
    print(totalgencost)
    model.optimize()
    model.write('model.mps')

    shedloads = np.zeros((T,NB))
    linestatus_out = np.zeros((T,NL))
    genvalues = np.zeros((T,NGENS))

    for t in range(0, T):
        for l in range(0,NL):
            linestatus_out[t][l]=linestatus[t,l].X
    for t in range(0, T):
        for gen in range(0,NGENS):
            genvalues[t][gen] = pgs[t,gen].X   
    
    #The function must exports the:
    #    1. shedloads: load_sheding(always a vector of zero)
    #    2. genvalues: the generation setpoints of each generator
    #    3. model: gurobipy model
    #    4. linestatus_out: the decided binary decisions
    return shedloads, genvalues, model,linestatus_out

In [ ]:
#Check the model and find the optimal solution
shedloads, genvalues,model,linestatus = runOPF()

#### Get information about the above problem
This step is crucial. This is where we start building a generalized framework that is able to transform any MILP to the proposed solution methodology. Specifically gurobi represents general MILPs as: 

\begin{equation} \label{first_form}
\begin{aligned}
\max_{\boldsymbol{x}} \quad &\mathbf{c}^{T} \boldsymbol{x} \\
    s.t. \quad &\mathbf{A} \boldsymbol{x} \leq \text{RHS} \\
       & \text{LB} \leq \boldsymbol{x} \leq \text{UB} \\
     & {\boldsymbol{x}} \in \mathbb{X}
\end{aligned} 
\end{equation}

In [ ]:
#Framework
#Master
RHS = np.array(model.getAttr('RHS'))
Sense = model.getAttr('Sense') 
LB = np.array(model.getAttr('LB'))
UB = np.array(model.getAttr('UB'))
VType = model.getAttr('VType')
A=model.getA().toarray()
VType=np.array(VType)
c = model.getAttr('Obj')
c = np.asarray(c)
names=np.array(model.getAttr('VarName'))
NConstraints=A.shape[0]
NVariables=A.shape[1]

Binary_variables_index=np.where(VType == 'B')
Continous_variables_index=np.where(VType == 'C')
# for
constraints_only_binaries=[]
constraints_only_continous=[]
constaints_mixed=[]

#find constraints variable types
for constr_ind in range(NConstraints):
    
    #3 Choices so 2 bools
    #If the constraint is mixed
    bool_mixed=False
    
    #if the constraint is only with integers
    bool_binaries=False
    
    for var_ind in range(NVariables):
        if A[constr_ind][var_ind]>0:
            if var_ind in Binary_variables_index[0]:
                bool_binaries=True
            if var_ind in Continous_variables_index[0]:
                bool_mixed=True
    if bool_binaries and bool_mixed:
        constaints_mixed.append(constr_ind)
    elif bool_binaries and bool_mixed==False:
        constraints_only_binaries.append(constr_ind)
    else:
        constraints_only_continous.append(constr_ind)
        
#We get the indexes of the constraints in A matrix that have only binary variables or continuous variables 
#or they are mixed 
constraints_only_binaries=np.array(constraints_only_binaries)
constraints_only_continous=np.array(constraints_only_continous)
constaints_mixed=np.array(constaints_mixed)


#### Create the Subproblem Problem parametrized with the Parameters above as:
\begin{align}
    \max_{\boldsymbol{y}} \quad &\mathbf{i}^{T} \boldsymbol{z}^{(n)} + \mathbf{c}^{T} \boldsymbol{y} \tag{SP} \label{sub_problem}\\
    s.t. \quad &\mathbf{B} \boldsymbol{y} \leq \mathbf{b} - \mathbf{A} \boldsymbol{z}^{(n)}  \quad :\boldsymbol{\lambda} \label{linear_constr}
\end{align}

We denote that here we formulate the primal subproblem as we can get the dual variables with a strait forward way using gurobipy. Therefore, when Subproblem is infeasible, its dual counterpart will be unbounded. This means that the dual variables represent a dual ray, which can be acquired by using the farkas lema.

the create_master function takes as arguments:
1. Binary_fixed: the binary decisions resulted by QUBO

In [ ]:
def create_Master(Binary_fixed,c):
    NConstraints=A.shape[0]
    NVariables=A.shape[1]
    
    model=gp.Model("Subproblem")
    
    variables = model.addVars(len(Continous_variables_index[0]),lb=-GRB.INFINITY,vtype=VType[0], name="variables")

    
    for constr in range(NConstraints):
         model.addConstr(gp.quicksum(A[constr][vari]*variables[i] for i,vari in enumerate(Continous_variables_index[0]))
                        +gp.quicksum(A[constr][vari]*Binary_fixed[i] for i,vari in enumerate(Binary_variables_index[0])) <= RHS[constr])


    obj=gp.quicksum(c[vari]*variables[i] for i,vari in enumerate(Continous_variables_index[0]))#+gp.quicksum(c[vari]*variables[i] for i,vari in enumerate(Binary_variables_index[0]))#-270

    model.setObjective(obj, GRB.MINIMIZE)#GRB.MINIMIZE)
    model.update()
    model.Params.InfUnbdInfo=1
    model.optimize()
    model.update()
    print(obj)

    extreme_rays=False
    extreme=[]
    obj=10000

    if model.Status == GRB.UNBOUNDED:
        print("ERRRRRR")
        return
    
    if model.Status == GRB.INFEASIBLE:
        obj=-10000
        extreme_rays=True
        for i,c in enumerate(model.getConstrs()):
            extreme.append(c.FarkasDual)
        print("NUMBER OF CONSTRAINTS",i)

        return extreme_rays,np.array(extreme),obj,model
    for c in model.getConstrs():
        extreme.append(c.Pi)
        
    obj = model.ObjVal
    
    return extreme_rays,extreme,obj,model
    

In [ ]:
#Test Subproblem problem
test=linestatus[0]
extreme_ray_bool,temp_extreme,LB_temp,mast=create_Master(test,c)

#### Create QUBO parametrized with the variables denoted before as:
\begin{align}
    \max_{\boldsymbol{z}} \quad & \mathbf{i}^{T} \boldsymbol{z} + s(\boldsymbol{p}) + ||{\boldsymbol{z}}-{\boldsymbol{z}}^{(n-1)}|| \tag{MP-Opt} \label{master_optimized}\\
 \quad s.t. \quad & \boldsymbol{p}^{t}(\mathbf{b} - \mathbf{A} \boldsymbol{z}) \geq s \quad \text{for } t \in T \notag \\
      \quad & \boldsymbol{r}^{k}(\mathbf{b} - \mathbf{A} \boldsymbol{z}) \geq 0 \quad \text{for } k \in K \notag
\end{align}

Function "create_qubo" gets as arguments:
1. extreme_rays: an array of dimensions (Num of extreme rays,Num of constraints)
2. extreme_points: an array of dimensions (Num of extreme rays,Num of constraints)
3. prev_points: an array of the decisions in the previous timeslot (intially z=vector(0))

and returns 
1. bins: the final binary decisions
2. obj: the resulting objective function
3. model: the resulting gurobipy model


In [ ]:
#optimized
def create_qubo(extreme_rays,extreme_points,prev_bins=0):
    extreme_rays=np.array(extreme_rays)
    extreme_points=np.array(extreme_points)
    
    #Denote how many qubits we would like to represent each part of s(p)
    acc=10

    NConstraints_rays=extreme_rays.shape[0]
    NConstraints_points=extreme_points.shape[0]
    print(NConstraints_rays,NConstraints_points)
    NVariables=A.shape[1]
    NConstraints_master=A.shape[0]
    model=GRB.Model("Subproblem")
    
    variables = model.addVars(len(Binary_variables_index[0]),vtype=GRB.BINARY, name="variables")

    eta=model.addVars(3,acc,vtype=GRB.BINARY, name="eta")
    
    model.addConstrs(GRB.quicksum(extreme_rays[constr_rays][constr]*RHS[constr]-extreme_rays[constr_rays][constr]*GRB.quicksum(A[constr][vari]*variables[i] for i,vari in enumerate(Binary_variables_index[0])) for constr in range(NConstraints_master)) >= 0 for constr_rays in range(NConstraints_rays))
    model.addConstr(GRB.quicksum(variables[i] for i,vari in enumerate(Binary_variables_index[0]))==5)


    model.addConstrs(GRB.quicksum(extreme_points[constr_points][constr]*RHS[constr]-extreme_points[constr_points][constr]*GRB.quicksum(A[constr][vari]*variables[i] for i,vari in enumerate(Binary_variables_index[0])) for constr in range(NConstraints_master)) <=
                     GRB.quicksum((2**i)*eta[0,i] for i in range(acc))+GRB.quicksum((2**(-i))*eta[1,i] for i in range(acc))-GRB.quicksum((2**i)*eta[2,i] for i in range(acc)) for constr_points in range(NConstraints_points))
    
    #Uncomment the commented part if you want to add the final term of the objective function the optimization shown above
    obj=GRB.quicksum((2**i)*eta[0,i] for i in range(acc))+GRB.quicksum((2**(-i))*eta[1,i] for i in range(acc))-GRB.quicksum((2**i)*eta[2,i] for i in range(acc))#+GRB.quicksum((variables[i]-prev_bins[i])*(variables[i]-prev_bins[i])for i in range(len(Binary_variables_index[0])))#+0.001*(GRB.quicksum((variables[i]-prev_bins[i])*(variables[i]-prev_bins[i])for i in range(len(Binary_variables_index[0]))))#+0.1GRB.quicksum(variables[i] for i,vari in enumerate(Binary_variables_index[0]))

    model.setObjective(obj,GRB.MINIMIZE)
    model.update()
    model.Params.InfUnbdInfo=1
    model.Params.Presolve=1
    model.optimize()
    model.update()
    
    bins=np.zeros(len(Binary_variables_index[0]))
    for i in range(len(Binary_variables_index[0])):
        bins[i]=variables[i].X
    print(bins)
    
    #Uncomment the commented part if you want to add the final term of the objective function the optimization shown above
    obj = model.ObjVal#-sum((variables[i].X-prev_bins[i])*(variables[i].X-prev_bins[i])for i in range(len(Binary_variables_index[0])))
    
    print(obj)
    return bins,obj,model


In [ ]:
#This function takes a model as input and transform it into a cqm format
def qubo_to_qpu(model_qubo):
    #Save model to actual LP file
    mod = QuadraticProgram()
    mod = from_gurobipy(model_qubo)
    #Save the lp file
    mod.write_to_lp_file("qubo1.lp")
    
    #Open lp file with dimod
    with open(r'qubo1.lp', 'rb') as f:
        cqm = dimod.lp.load(f)
    return cqm


In [ ]:
#This function takes a gurobipy model as input and solves it with a quantum computer.
#Where NL+1, the number interpretation is number of binary decision variables+1
def get_qubo_dwave_all_solution(model_qubo):
    solvable_qubo=qubo_to_qpu(model_qubo)
    
    #Replace the given with your token
    cqm_sampler=LeapHybridCQMSampler(token="REPLACE_with_your_token")
    
    #Run the algorithms on the Quantum computer and save the results into variable sampleser
    sampleser=cqm_sampler.sample_cqm(solvable_qubo,time_limit=5)
    
    #Find the subset of the solutions that were feasible
    feasible_sampleset = sampleser.filter(lambda row: row.is_feasible)
    
    #get the number of feasible solutions found
    tot_solutions=len(feasible_sampleset)
    
    #Get total QPU time
    qpu_time=sampleser.info['qpu_access_time']/10**6
    total_time=sampleser.info['run_time']/10**6
    
    #find the best solution
    best = feasible_sampleset.first
    
    #Find all the solutions that gave the lower energy
    best_solutions=list(feasible_sampleset.lowest(atol=.0000000000000001).samples())
    
    #Get the G best solutions
    G=3 #Change that for returning a different number of solutions
    fil_sols=[]
    for j in best_solutions:
        sol=[]
        best=j
        selected_item_indices = [(int(''.join(filter(str.isdigit, key))),val) for key, val in best.items() if int(''.join(filter(str.isdigit, key)))<=NL+1]

        for i in range(NL+1):
            for (key,val) in selected_item_indices:
                if key==i:
                    sol.append(val)
        fil_sols.append(sol)
    
    if len(fil_sols)>G:
        fil_sols=fil_sols[:G]
    return np.array(fil_sols), float(feasible_sampleset.first.energy), qpu_time, total_time,tot_solutions



In [ ]:
#Initialize variables
extreme_rays=[]
extreme_points=[]
extreme_rays_opt=[]
extreme_opt=[]

UB_benders= 10000
LB_benders= -10000
lbs=[]
ubs=[]
iteration=1
bins_fixed=np.array([np.zeros(len(Binary_variables_index[0]))])
prev_bins=np.zeros(len(Binary_variables_index[0]))
pareto_bins=np.zeros(len(Binary_variables_index[0]))
sols=[]
qpu_times=[]
time_neededs=[] 
cont=True
tme=0

#Start the decomposed algorithm
while(np.abs((UB_benders-LB_benders))>0.1 and cont):
    
    print("Iteration ",iteration)
    saved_LBs=[]
    saved_LB_solutions=[]
    print("Executing subproblem")
    
    #For all the found solutions run their Subproblem
    for i in bins_fixed:
        print(i)
        extreme_ray_bool,_,LB_temp,mast=create_Master(np.array(i),c)
        LB_benders=LB_temp#min(LB_temp,LB_benders)
        print(LB_temp)
        print("FOUND LB",LB_benders)
        print("###################HERE##################", (UB_benders-LB_benders))
        
        #If you found a solution stop the algorithm
        if np.abs((UB_benders-LB_benders))<0.1:
            cont=False
            break
        
        #Approximate the pareto point
        pareto_bins=1/2*pareto_bins+1/2*i
        
        mast1=0

        #Run the pareto problem
        #If you do not want this acceleration method comment out the commands between the next to print commands
        print("##############SOLVE PARETO###############")
        extreme_ray_bool,temp_extreme,_,mast1=create_Master(pareto_bins,c)
        print("##############SOLVE PARETO###############")
        saved_LB_solutions.append(i)
        saved_LBs.append(LB_temp)
        
        #save all the lower bound solutions
        lbs.append(LB_temp)
        print()
        print("FOUND LB",LB_benders)
        bool_add=False
        tme+=mast1.Runtime+mast.Runtime
        if extreme_ray_bool or bool_add:
            extreme_rays.append(temp_extreme)

        else:
            sols.append(bins_fixed)
            extreme_points.append(temp_extreme)
    print("Executing qubo")
    
    #If you did not found a solution, run the qubo in the quantum computer again
    if np.abs((UB_benders-LB_benders))>0.1 and cont:
        bins_fixed,UB_benders,model_qubo=create_qubo(extreme_rays,extreme_points,prev_bins)
        bins_fixed,UB_benders,qpu_time,time_needed,tot_solutions=get_qubo_dwave_all_solution(model_qubo)#get_qubo_dwave_solution(model_qubo)
        
    print("SOLUTIONS..........",tot_solutions)
    prev_bins=bins_fixed.copy()
    qpu_times.append(qpu_time)
    time_neededs.append(time_needed)
    print()
    ubs.append(UB_benders)
    print("FOUND UB",UB_benders)
    if len(saved_LB_solutions)>0:
        print(saved_LB_solutions[-1])
        
    #Find times and increase iteration counter
    tme+=qpu_time
    iteration+=1